# 🤖 AI Face Emotion Detection - Model Retraining

This notebook retrains the emotion detection model using user feedback data collected from the app.

## Workflow:
1. **Load Feedback Data** - From CSV and images
2. **Visualize Data** - Check what was corrected
3. **Fine-tune Model** - Retrain with feedback
4. **Evaluate Performance** - Compare old vs new
5. **Update Model** - Replace original with improved version

In [ ]:
# Import Required Libraries
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

## 📁 Configuration

Set up paths and emotion labels.

In [ ]:
# Configuration
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
FEEDBACK_CSV = os.path.join(BASE_DIR, "feedback_log.csv")
FEEDBACK_IMAGES_DIR = os.path.join(BASE_DIR, "feedback_images")
MODEL_PATH = os.path.join(BASE_DIR, "best_emotion_model.keras")
RETRAINED_MODEL_PATH = os.path.join(BASE_DIR, "best_emotion_model_retrained.keras")

EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

print(f"📂 Feedback CSV: {FEEDBACK_CSV}")
print(f"📂 Images Directory: {FEEDBACK_IMAGES_DIR}")
print(f"📂 Model Path: {MODEL_PATH}")

## 📊 Load & Analyze Feedback Data

Load the feedback data and visualize what users corrected.

In [ ]:
# Load feedback CSV
if not os.path.exists(FEEDBACK_CSV):
    print("❌ No feedback data found. Run the app first to collect feedback.")
else:
    df = pd.read_csv(FEEDBACK_CSV)
    
    # Clean data
    df = df.dropna(subset=['image_path', 'corrected_emotion'])
    
    print(f"✅ Found {len(df)} feedback records\n")
    display(df)

### 📈 Visualize Feedback Statistics

In [ ]:
# Visualize feedback statistics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Predicted vs Corrected Emotions
ax1 = axes[0, 0]
emotion_counts = df['corrected_emotion'].value_counts()
emotion_counts.plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('Corrected Emotion Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Emotion', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.tick_params(axis='x', rotation=45)

# 2. Correction matrix (predicted -> corrected)
ax2 = axes[0, 1]
correction_matrix = pd.crosstab(df['predicted_emotion'], df['corrected_emotion'])
sns.heatmap(correction_matrix, annot=True, fmt='d', cmap='YlOrRd', ax=ax2, cbar_kws={'label': 'Count'})
ax2.set_title('Prediction Correction Matrix', fontsize=14, fontweight='bold')
ax2.set_xlabel('Corrected Emotion', fontsize=12)
ax2.set_ylabel('Predicted Emotion', fontsize=12)

# 3. Confidence distribution
ax3 = axes[1, 0]
df['confidence'].hist(bins=20, ax=ax3, color='lightgreen', edgecolor='black')
ax3.set_title('Confidence Score Distribution', fontsize=14, fontweight='bold')
ax3.set_xlabel('Confidence', fontsize=12)
ax3.set_ylabel('Frequency', fontsize=12)
ax3.axvline(df['confidence'].mean(), color='red', linestyle='--', label=f'Mean: {df["confidence"].mean():.2f}')
ax3.legend()

# 4. Feedbacks over time
ax4 = axes[1, 1]
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.groupby(df['timestamp'].dt.date).size().plot(kind='line', marker='o', ax=ax4, color='purple')
ax4.set_title('Feedback Collection Over Time', fontsize=14, fontweight='bold')
ax4.set_xlabel('Date', fontsize=12)
ax4.set_ylabel('Number of Feedbacks', fontsize=12)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📊 Summary Statistics:")
print(f"   Total feedbacks: {len(df)}")
print(f"   Average confidence: {df['confidence'].mean():.4f}")
print(f"   Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

### 🖼️ Display Sample Feedback Images

View some of the images that were corrected.

In [ ]:
# Display sample feedback images
num_samples = min(6, len(df))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx in range(num_samples):
    row = df.iloc[idx]
    image_path = os.path.join(FEEDBACK_IMAGES_DIR, row['image_path'])
    
    if os.path.exists(image_path):
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        axes[idx].imshow(img, cmap='gray')
        axes[idx].set_title(f"Predicted: {row['predicted_emotion']}\nCorrected: {row['corrected_emotion']}", 
                           fontsize=10, fontweight='bold')
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, 'Image Not Found', ha='center', va='center')
        axes[idx].axis('off')

# Hide empty subplots
for idx in range(num_samples, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 🔄 Load Feedback Data for Training

Prepare images and labels for model retraining.

In [ ]:
# Load feedback data
X = []  # Images
y = []  # Labels

for idx, row in df.iterrows():
    image_filename = str(row['image_path']).strip()
    correct_emotion = str(row['corrected_emotion']).strip()
    
    image_path = os.path.join(FEEDBACK_IMAGES_DIR, image_filename)
    
    if os.path.exists(image_path):
        # Load image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = img / 255.0  # Normalize
            X.append(img)
            y.append(EMOTION_LABELS.index(correct_emotion))
            print(f"  ✅ Loaded: {image_filename} → {correct_emotion}")
        else:
            print(f"  ❌ Failed to load: {image_filename}")
    else:
        print(f"  ❌ Image not found: {image_path}")

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Reshape for model input (batch, height, width, channels)
X = X.reshape(-1, 48, 48, 1)

print(f"\n📊 Data Summary:")
print(f"   Total samples: {len(X)}")
print(f"   Shape: {X.shape}")
print(f"   Labels shape: {y.shape}")
print(f"   Min/Max pixel values: {X.min():.2f} / {X.max():.2f}")

## 🧠 Load Original Model

Load the current model that will be fine-tuned.

In [ ]:
# Load original model
print("🔄 Loading original model...")
model = load_model(MODEL_PATH)

print("\n📈 Model Architecture:")
model.summary()

## 🚀 Fine-Tune Model with Feedback Data

Retrain the model with data augmentation to improve generalization.

In [ ]:
# Compile model with lower learning rate for fine-tuning
optimizer = Adam(learning_rate=0.0001)  # Lower learning rate for fine-tuning
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Data augmentation to improve generalization
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

print("🚀 Starting fine-tuning...")
print(f"   Epochs: 10")
print(f"   Batch size: 8")
print(f"   Samples: {len(X)}")
print(f"   Learning rate: 0.0001\n")

# Fine-tune with the feedback data
history = model.fit(
    datagen.flow(X, y, batch_size=8),
    epochs=10,
    verbose=1,
    steps_per_epoch=max(1, len(X) // 8)
)

print("\n✅ Fine-tuning complete!")

## 📊 Visualize Training Results

Plot the training loss and accuracy curves.

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1 = axes[0]
ax1.plot(history.history['loss'], marker='o', linewidth=2, color='red')
ax1.set_title('Training Loss Over Epochs', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2 = axes[1]
ax2.plot(history.history['accuracy'], marker='o', linewidth=2, color='green')
ax2.set_title('Training Accuracy Over Epochs', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Final Training Results:")
print(f"   Final Loss: {history.history['loss'][-1]:.4f}")
print(f"   Final Accuracy: {history.history['accuracy'][-1]:.4f}")

## 💾 Save Retrained Model

Save the improved model.

In [ ]:
# Save the retrained model
model.save(RETRAINED_MODEL_PATH)
print(f"✅ Retrained model saved to:\n   {RETRAINED_MODEL_PATH}")

## 🔄 Replace Original Model (Optional)

This will backup the old model and replace it with the new one.

**⚠️ Warning:** This will update the model used by your Streamlit app!

In [ ]:
# Backup and replace original model
import shutil

REPLACE_MODEL = True  # Set to True to replace, False to keep original

if REPLACE_MODEL:
    backup_path = os.path.join(BASE_DIR, "best_emotion_model_backup.keras")
    
    if os.path.exists(RETRAINED_MODEL_PATH):
        # Backup original model
        if os.path.exists(MODEL_PATH):
            shutil.copy(MODEL_PATH, backup_path)
            print(f"💾 Original model backed up to:\n   {backup_path}")
        
        # Replace with retrained model
        shutil.copy(RETRAINED_MODEL_PATH, MODEL_PATH)
        print(f"\n✨ Model updated successfully!")
        print(f"   New model: {MODEL_PATH}")
        print(f"\n✅ Your app will now use the improved model!")
    else:
        print("❌ Retrained model not found!")
else:
    print("⚠️ Original model unchanged.")
    print(f"   Retrained model saved as: {RETRAINED_MODEL_PATH}")
    print("\n   Set REPLACE_MODEL = True to update the active model.")

## 🎯 Summary

### What We Did:
1. ✅ Loaded {len(df)} feedback records from users
2. ✅ Visualized correction patterns and statistics
3. ✅ Fine-tuned the model with feedback data
4. ✅ Saved the improved model

### Next Steps:
1. Run your Streamlit app to test the improved model
2. Continue collecting feedback to further improve accuracy
3. Retrain periodically (weekly/monthly) as more feedback accumulates

### Model Files:
- **best_emotion_model.keras** - Active model (used by app)
- **best_emotion_model_backup.keras** - Previous version
- **best_emotion_model_retrained.keras** - Latest retrained version

---

**🚀 Happy Training!**